In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import os
import pandas as pd
import numpy as np
import sys
from tqdm import tqdm
sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import dlc_utils
from matplotlib import pyplot as plt

In [3]:
from importlib import reload
reload(dlc_utils)

<module 'dlc_utils' from '/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis/dlc_utils.py'>

In [4]:
dlc_directory = '/volumes/My_Passport/dlc_analysis/behavcamvideos/rotarod/3_19_2020/'

In [12]:
experiment_folders = [folder for folder in os.listdir(dlc_directory) if '_M' in folder][-3:]

In [13]:
experiment_folders

['H16_M33_S20', 'H16_M39_S59', 'H16_M4_S33']

In [14]:
# GRIN018_H17_M41_S43
#experiment_folders = ['H19_M46_S0']

In [15]:
for folder in tqdm(experiment_folders):
    output_directory = dlc_directory+folder+'/'
    session_name = folder
    print(folder)
    dlc_output_files = os.listdir(output_directory)
    dlc_output_files.sort()
    
    names = []
    dlc_dfs = []
    coordinates_dfs = []
    for output_file in dlc_output_files:
        if output_file.endswith('000.h5'):
            print(output_file)
            try:
                dlc_output = pd.read_hdf(output_directory + output_file)
                dlc_output = dlc_utils.calculate_centroid(dlc_output)
                dlc_output.head()
                coordinates_delta_df = dlc_utils.difference_df(dlc_output)
                df_columns = list(dlc_output.columns)
                print('calculating velocity')
                velocity_df = pd.DataFrame(np.transpose(np.array([np.array([dlc_utils.velocity(coordinates_delta_df[body_part]['x'].values[frame],coordinates_delta_df[body_part]['y'].values[frame]) for frame in range(len(coordinates_delta_df))]) for body_part in list(set([df_columns[item][1] for item in range(len(df_columns))]))])), 
                                       columns=list(set([df_columns[item][1] for item in range(len(df_columns))]))) 
                for body_part in list(set([df_columns[item][1] for item in range(len(df_columns))])):
                    velocity_df[body_part+'_likelihood']=dlc_output[df_columns[0][0]][body_part]['likelihood'].values
            
                names.append(output_file)
                dlc_dfs.append(velocity_df)
                coordinates_dfs.append(dlc_output)
            except ValueError:
                print('not_found:' + str(output_file))
    try:        
        session_compiled = pd.concat(dlc_dfs, keys=names)
        coordinates_dfs_compiled = pd.concat(coordinates_dfs, keys=names)

    #save compiled session 
        session_compiled.to_hdf(output_directory+session_name+'_compiled_session_04142020.h5', key='df')
        coordinates_dfs_compiled.to_hdf(output_directory+session_name+'_compiled_coordinates_session_04142020.h5', key='df')
        
    except ValueError:
        print('no h5 files for:' + str(output_file))
    



  0%|          | 0/3 [00:00<?, ?it/s]

H16_M33_S20
no h5 files for:behavCam1DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
H16_M39_S59
behavCam1DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5




 67%|██████▋   | 2/3 [00:00<00:00,  6.01it/s]

not_found:behavCam1DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
no h5 files for:behavCam1DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
H16_M4_S33
behavCam1DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam2DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity
behavCam3DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000.h5
calculating velocity




100%|██████████| 3/3 [01:07<00:00, 22.54s/it]


In [ ]:
    #create behavior data aligned to the msCam frames
    aligned = dlc_utils.downsample_dlc_to_behavior(output_directory+session_name+'_compiled_session_04142020.h5', output_directory+'timestamp.dat')
    aligned.reset_index(inplace=True)
    aligned['msCamFrame'] = aligned.index.values
    aligned = aligned.set_index(pd.to_timedelta(np.linspace(0, len(aligned)*(1/20), len(aligned)), unit='s'), drop=False)

    #do same for coordinates df 
    coordinates_aligned = dlc_utils.downsample_dlc_to_behavior(output_directory+session_name+'_compiled_coordinates_session_04142020.h5', output_directory+'timestamp.dat')
    coordinates_aligned.reset_index(inplace=True)
    coordinates_aligned['msCamFrame'] = coordinates_aligned.index.values
    coordinates_aligned = coordinates_aligned.set_index(pd.to_timedelta(np.linspace(0, len(coordinates_aligned)*(1/20), len(coordinates_aligned)), unit='s'), drop=False)
    
    #smooth data by downsampling and interpolating 
    interpolated = dlc_utils.downsample_and_interpolate(aligned, '.05S', '1S', 'linear')
    interpolated.to_csv(output_directory+session_name+'_dlc_tracking_foranalysis_04142020.csv')

    coordinates_aligned_interpolated = dlc_utils.downsample_and_interpolate(coordinates_aligned, '.05S', '1S', 'linear')
    coordinates_aligned_interpolated.to_csv(output_directory+session_name+'_dlc_coordinates_smoothed_tracking_foranalysis_04142020.csv')
    
    coordinates_aligned.to_csv(output_directory+session_name+'_dlc_coordinates_tracking_foranalysis_04142020.csv')

In [ ]:
coordinates_dfs_compiled = pd.concat(coordinates_dfs, keys=names)

In [ ]:
coordinates_dfs_compiled

In [ ]:
coordinates_aligned = dlc_utils.downsample_dlc_to_behavior(output_directory+session_name+'_compiled_coordinates_session_04142020.h5', output_directory+'timestamp.dat')

In [ ]:
coordinates_aligned

In [ ]:
coordinates_aligned['msCamFrame'] = coordinates_aligned.index.values

In [ ]:
interpolated = dlc_utils.downsample_and_interpolate(aligned, '.05S', '1S', 'linear')

In [ ]:
coordinates_aligned = coordinates_aligned.set_index(pd.to_timedelta(np.linspace(0, len(coordinates_aligned)*(1/20), len(coordinates_aligned)), unit='s'), drop=False)

In [ ]:
coordinates_aligned_interpolated = dlc_utils.downsample_and_interpolate(coordinates_aligned, '.05S', '1S', 'linear')

In [ ]:
coordinates_aligned_interpolated

In [ ]:
plt.plot(coordinates_aligned_interpolated['DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000']['right_r_paw']['x'].values)

In [ ]:
plt.plot(coordinates_aligned['DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000']['right_r_paw']['y'].values)

In [ ]:
plt.plot(coordinates_aligned['DLC_resnet50_GRIN_rotarod_rearviewApr2shuffle1_300000']['right_r_paw']['x'].values)

In [ ]:
interpolated = dlc_utils.downsample_and_interpolate(aligned, '.05S', '1S', 'linear')

In [ ]:
interpolated

In [ ]:
plt.plot(interpolated['tail_base_likelihood'].values)

In [ ]:
plt.plot(interpolated['tail_base'].values)

In [ ]:
plt.plot(interpolated['right_r_paw_likelihood'].values)

In [ ]:
plt.plot(interpolated['right_r_paw'].values)

In [ ]:
#aligned = dlc_utils.downsample_dlc_to_behavior(output_directory+'GRIN012_H17_M46_S35_compiled_session_04142020.h5', output_directory+'timestamp.dat')
dlc_analysis = pd.read_hdf(output_directory+'GRIN012_H17_M46_S35_compiled_session_04142020.h5')

In [ ]:
frame_clock_df = pd.read_table(output_directory+'timestamp.dat')

In [ ]:
dlc_full = dlc_full.reset_index()

In [ ]:
msCam_timestamps = frame_clock_df[frame_clock_df['camNum'] == 0].set_index('frameNum')
behavCam_timestamps = frame_clock_df[frame_clock_df['camNum'] == 1].set_index('frameNum')
msCam_timestamps['sysClock'][1] = 0
behavCam_timestamps['sysClock'][1] = 0

In [ ]:
msCam_timestamps

In [ ]:
behavCam_timestamps

In [ ]:
msCam_timestamps = dlc_utils.align_behavior_data(msCam_timestamps, behavCam_timestamps)
msCam_timestamps.reset_index(inplace=True)